In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import tqdm 

In [3]:
from sklearn.metrics import f1_score, precision_recall_curve


In [13]:
subsets = ['validate','test']
threshold = 0.5

for subset in subsets: 
    result_file = '../BERT/BERT-{}-results-split5.txt'.format(subset)
    fp = open(result_file)
    lines = fp.readlines() 
    lines = [x.strip() for x in lines]
    fp.close()
    print(lines[:10])
    
    y_true = []
    y_pred = [] 
    y_prob = [] 
    
    covid_ids = [] 
    non_covid_ids = [] 

    for l in lines: 
        l_splits = l.split(',') 
        #print(l_splits)
        
        #change covid class label to 1 
        y_true.append(int(l_splits[1]))    
        
        prob0 = float(l_splits[3])
        prob1 = float(l_splits[4])

        score = np.exp(prob0)/(np.exp(prob0)+np.exp(prob1))        
        pred2 = (score < threshold).astype(np.uint8)
        
        #y_pred.append(int(l_splits[2])) 
        y_pred.append(pred2) 
        y_prob.append(score)
        
        if pred2==0:
            covid_ids.append(l_splits[0])
        else: 
            non_covid_ids.append(l_splits[0])
            
    y_pred = np.asarray(y_pred)
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)

    F1_1 = f1_score(y_true, y_pred, average='micro')
    F1_2 = f1_score(y_true, y_pred, average='macro')
    print(F1_1,F1_2)
    
    fp = open('BERT-{}-{}-covid.csv'.format(threshold,subset),'w')    
    len_covid = len(covid_ids) 
    for i in range(len_covid): 
        ids = covid_ids[i].split(',')[0].split('_')[-1].split('.')[0]        
        if i<len_covid-1:
            fp.write('{},'.format(ids))
        else:
            fp.write('{}'.format(ids))
    fp.close() 

    fp = open('BERT-{}-{}-non-covid.csv'.format(threshold,subset),'w')
    len_non_covid = len(non_covid_ids) 
    for i in range(len_non_covid):
        ids = non_covid_ids[i].split(',')[0].split('_')[-1].split('.')[0]
        if i<len_non_covid-1:
            fp.write('{},'.format(ids))
        else:
            fp.write('{}'.format(ids))
    fp.close() 
    
    

['val/covid/ct_scan_0,0,0,2.545,-2.029', 'val/covid/ct_scan_1,0,0,2.522,-2.174', 'val/covid/ct_scan_2,0,0,1.184,-1.170', 'val/covid/ct_scan_3,0,0,2.243,-1.944', 'val/covid/ct_scan_4,0,0,2.168,-1.934', 'val/covid/ct_scan_5,0,0,1.935,-1.823', 'val/covid/ct_scan_6,0,0,2.751,-2.275', 'val/covid/ct_scan_7,0,0,2.886,-2.336', 'val/covid/ct_scan_8,0,0,2.409,-2.249', 'val/covid/ct_scan_9,0,0,1.933,-1.656']
0.9144385026737968 0.9134484653880645
['test/subset1/3ea85c05-2cd1-4b5a-822f-3985b0e06867,0,1,-0.853,0.927', 'test/subset1/2c442cd2-0abd-4ebd-9223-ff7ee8e9953c,0,1,-4.329,4.130', 'test/subset1/3c5a2a48-0a9b-42f1-9e1c-e90fe209d179,0,1,-2.163,1.977', 'test/subset1/0c000b69-41de-41cd-b498-37d993c94366,0,1,-0.690,0.361', 'test/subset1/4a81f8d8-e255-44ff-81d1-717868e7b381,0,1,-2.243,2.251', 'test/subset1/4a1509c1-a588-4f95-a167-96a724f2ce22,0,0,0.771,-0.432', 'test/subset1/2f9fe28a-9c06-4604-847a-4830163650f6,0,0,0.202,-0.304', 'test/subset1/03c094cc-b595-4fc9-b21d-66a7d5d7bee9,0,1,-1.902,1.904', 